In [1]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities,lambdas
import numpy as np
from util import welfare_metrics,student_by_col,bayes_update_grade,grades_gr_to_grades_col
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est
from scipy.optimize import fsolve
from scipy.stats import norm, multivariate_normal

In [2]:
print(capacities_rate)

[0.2, 0.2]


In [3]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [4]:
def cdf(x, sigma):
    return norm.cdf(x, scale = sigma)

In [5]:
def cdfm(x, y, sigma, rho):
    return multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [6]:
def cdfmsi(x, y, sigma, rho):
    return norm.cdf(y, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [7]:
def cdfmis(x, y, sigma, rho):
    return norm.cdf(x, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [8]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [9]:
def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
    return fsolve(func, x0 = np.array([0, 0]))

In [10]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = chi[0]**2/sigma_i**2
cor_ii = chi[1]**2/sigma_ii**2

In [11]:
print(cor_i,cor_ii)

0.4999999999999999 0.4999999999999999


In [12]:
P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [13]:
cutoff_values = [P_A,P_B]

In [14]:
P_A,P_B

(np.float64(1.6610871399250278), np.float64(1.8869875908985916))

In [15]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.70
Proportion of students in group 0 with only a second preference offer 0.07
Proportion of students in group 0 with first choice offer 0.23
Proportion of students in group 1 with no offer 0.68
Proportion of students in group 1 with only a second preference offer 0.09
Proportion of students in group 1 with first choice offer 0.23


In [16]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 863
Number of student admitted to college 2 in group 0: 923
Number of student admitted to college 1 in group 1: 1424
Number of student admitted to college 2 in group 1: 1420


(2287, 2343)

In [17]:
print(capacities)

[2976, 2976]


In [18]:
new_cutoff_values = [1.176513,0.10931724]
new_cutoff_values

[1.176513, 0.10931724]

In [19]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,chi,sigma,lambdas,bayes_type='right')

In [22]:
##    
# grade_estimated should have the following form 
# [[[-3.62656622994722, 2.1811620120023885, 
# -11.563939588668866, 9.522766463940787], 
# [-2.140950614292475, -3.1545339753546977, 
# 0.2686527056082677, -0.6574626041532636, 
# -2.252868772168457, 1.8937660992816987]], 
# [[-5.268780813142571, -0.8191620654894043, 
# -9.60125782299109, 7.9126466724151605], 
# [-3.9756947771758844, -2.6409442009387023, 
# 4.086647894727917, -1.357128391535272, 
# 0.8661128454577303, 0.9586986061854232]]] 
# with 2 groups, 2 colleges, group 1 has 4 students, 
# group 2 has 6 students.
# updated_grade_estimate should have the same form as grade_estimated

In [20]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.62
Proportion of students in group 0 with only a second preference offer 0.08
Proportion of students in group 0 with first choice offer 0.30
Proportion of students in group 1 with no offer 0.60
Proportion of students in group 1 with only a second preference offer 0.10
Proportion of students in group 1 with first choice offer 0.30


In [21]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1062
Number of student admitted to college 2 in group 0: 1227
Number of student admitted to college 1 in group 1: 1715
Number of student admitted to college 2 in group 1: 1865


(2777, 3092)